In [4]:
# 原先的代码已经在 tensorflow 2.0 中无法运行，但是库要求 tensorflow 2.1，所以明显有问题
# 因此不填空，直接重写。写一个基于 tensorflow 2.1 的 CNN 模型，实现 MNIST 分类任务

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 超参数
learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 2000

# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 归一化
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# 定义权重和偏置初始化函数
def weight_variable(shape):
    return tf.Variable(tf.random.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

# 定义卷积和池化函数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义模型
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, (7, 7), activation='relu', padding='same')
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2), padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same')
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2), padding='same')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dropout = tf.keras.layers.Dropout(1 - keep_prob_rate)
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.fc2(x)
        return x

# 创建模型
model = CNNModel()

# 定义损失函数和优化器
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate)

# 定义准确率计算函数
def compute_accuracy(v_xs, v_ys):
    y_pre = model(v_xs, training=False)
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy.numpy()

# 训练模型
for epoch in range(max_epoch):
    # 随机抽取一个批次
    idx = tf.random.shuffle(tf.range(len(x_train)))[:100]
    batch_xs, batch_ys = tf.gather(x_train, idx), tf.gather(y_train, idx)

    # 前向传播和反向传播
    with tf.GradientTape() as tape:
        predictions = model(batch_xs, training=True)
        loss = loss_fn(batch_ys, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 每 100 次迭代打印一次准确率
    if epoch % 100 == 0:
        acc = compute_accuracy(x_test[:1000], y_test[:1000])
        print(f"Epoch {epoch}, Accuracy: {acc}")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0, Accuracy: 0.14800000190734863
Epoch 100, Accuracy: 0.8659999966621399
Epoch 200, Accuracy: 0.9079999923706055
Epoch 300, Accuracy: 0.9399999976158142
Epoch 400, Accuracy: 0.9449999928474426
Epoch 500, Accuracy: 0.9539999961853027
Epoch 600, Accuracy: 0.9629999995231628
Epoch 700, Accuracy: 0.968999981880188
Epoch 800, Accuracy: 0.9739999771118164
Epoch 900, Accuracy: 0.9769999980926514
Epoch 1000, Accuracy: 0.9760000109672546
Epoch 1100, Accuracy: 0.9800000190734863
Epoch 1200, Accuracy: 0.9810